Ajout d'une colonne Id_p dans la table de travail T_complete pour remplacer Pays
et insertion des id_p d'apres T_Pays

In [ ]:
import psycopg2
import os
import csv
import pandas as pd

def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")


In [ ]:
conn=connexion_DB("santa_data")
cursor = conn.cursor()

In [ ]:
#ajout d'une colonne id_p dans T_complete
try:
    if(conn):
        req_modif='''
                ALTER TABLE T_complete 
                ADD COLUMN id_p int;
                '''
        cursor.execute(req_modif)
    
except(Exception, psycopg2.Error) as error :
    print ("Erreur lors de la modification de table T_complete", error)
    

In [ ]:
# mise a jour de T_complete par insertion des valeurs de id_p d'apres la T_pays
try:
    if(conn):     
        rq_maj='''UPDATE T_complete AS TC
                SET id_p= TP.ID_p
                FROM T_pays AS TP
                WHERE TC.Pays = TP.Pays;
                '''
        cursor.execute(rq_maj)
        conn.commit()
        
        rq_select='''
                SELECT ID , nom_lettre ,TC.id_p, TC.Pays ,TP.Pays , TP.ID_p
                FROM T_complete TC, T_pays TP
                WHERE TC.id_p=TP.ID_p;
                '''
        cursor.execute(rq_select)
        res=cursor.fetchall()
        print('\nTC_id'.ljust(8),'TC_pays'.ljust(20), 'TP_pays'.ljust(20),'TP_id'.ljust(8))
        trait="-"*45
        print(trait)
        for row in res:
            print(str(row[2]).ljust(8),end='')
            print(str(row[3]).ljust(20),end='')
            print(str(row[4]).ljust(20),end='')
            print(str(row[5]).ljust(8))
        
        
except(Exception, psycopg2.Error) as error :
    print ("Erreur lors de la maj de la table T_complete ", error)
    
finally:
    #closing database connection.
    if(conn):
        cursor.close()
        deconnexion_DB(conn)
 